# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [19]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 3
ROUNDS = 3
OTHER_DATE = '2023-10-27'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = [noise * 2 for noise in NOISE_LIST]

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


# Loop function

In [20]:
from tqdm import tqdm
import json
import cpp_soft_info
import pymatching

def func(distance, rounds, resets=False, shots = 1e5, p_meas = -1):
    # Initialize simulator
    simulator = RepCodeIQSimulator(provider, distance, rounds, DEVICE, _is_hex=_is_hex, _resets = resets, other_date=OTHER_DATE)
    IQ_data= simulator.generate_IQ(int(shots), noise_list=NOISE_LIST, verbose=False)
    print("Generated IQ data")

    # Run decoding
    result_grid = cpp_soft_info.decode_IQ_fast(simulator.stim_circ.detector_error_model(decompose_errors=False), IQ_data,
                                        rounds, int(LOGICAL), resets, simulator.qubit_mapping, simulator.grid_dict,
                                        simulator.processed_scaler_dict, _detailed=False, nb_intervals=-1, interval_offset=-1)

    result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(simulator.stim_circ.detector_error_model(decompose_errors=False), IQ_data,
                                           rounds, int(LOGICAL), resets, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=False, _ntnn_edges = True)

    return result_grid, result_informed


In [21]:
import numpy as np

distances = np.linspace(5, 55, 15)
distances

array([ 5.        ,  8.57142857, 12.14285714, 15.71428571, 19.28571429,
       22.85714286, 26.42857143, 30.        , 33.57142857, 37.14285714,
       40.71428571, 44.28571429, 47.85714286, 51.42857143, 55.        ])

In [35]:
import numpy as np
from tqdm import tqdm

shots = int(2.5e5) 
p_meas = -1

i_10 = True
i_20 = True
i_30 = True
i_40 = True
i_55 = True
# i_30 = False
# i_40 = False
# i_55 = False

for i in tqdm(distances, desc='Simulating for distance'):

    # Save results
    with open(f'../results/soft_vs_informed_simul_v2.json', 'r') as f:
        data = json.load(f)


    i = int(i)
    _redecode = False

    if i>=10 and i_10 == True:
        distance = 10
        rounds = 10
        _redecode = True
        i_10 = False

    if i>=20 and i_20 == True:  
        distance = 20
        rounds = 20
        _redecode = True
        i_20 = False

    if i >= 30 and i_30 == True:
        distance = 30
        rounds = 35
        _redecode = True
        i_30 = False
    
    if i >=40 and i_40:
        distance = i
        rounds = 35
        _redecode = True
        i_40 = False

    if i>=55 and i_55 == True:
        distance = i
        rounds = 35
        _redecode = True
        i_55 = False

    if _redecode:
        # Decode
        result_grid, result_informed = func(distance, rounds, shots = int(4.5e5), p_meas=p_meas)

        if str((distance, rounds)) not in data.keys():
            result_grid, result_informed = func(distance, rounds, shots = shots)
            data[str((distance, rounds))] = {'grid': result_grid.num_errors, 'informed': result_informed.num_errors, 'shots': shots, 'p_meas': p_meas}

        with open(f'../results/soft_vs_informed_simul_v2.json', 'w') as f:
            json.dump(data, f, indent=4)


   
    distance = i
    rounds = i     

    
    if str((distance, rounds)) not in data.keys():
        # Decode
        result_grid, result_informed = func(distance, rounds, shots = shots, p_meas=p_meas)
        data[str((distance, rounds))] = {'grid': result_grid.num_errors, 'informed': result_informed.num_errors, 'shots': shots, 'p_meas': p_meas}
    
    with open(f'../results/soft_vs_informed_simul_v2.json', 'w') as f:
        json.dump(data, f, indent=4)


Simulating for distance:   0%|          | 0/15 [00:00<?, ?it/s]

generated counts
correcting counts
finished correcting counts
Generated IQ data
generated counts
correcting counts
finished correcting counts
Generated IQ data


Simulating for distance:  20%|██        | 3/15 [02:17<09:10, 45.88s/it]

generated counts
correcting counts
finished correcting counts
Generated IQ data


Simulating for distance:  27%|██▋       | 4/15 [04:06<12:09, 66.31s/it]

...

Simulating for distance:  87%|████████▋ | 13/15 [2:41:58<40:52, 1226.08s/it]  

generated counts
correcting counts
finished correcting counts
Generated IQ data


Simulating for distance:  93%|█████████▎| 14/15 [3:05:51<21:28, 1288.24s/it]

generated counts
correcting counts
finished correcting counts
Generated IQ data
generated counts
correcting counts
finished correcting counts
Generated IQ data
generated counts
correcting counts
finished correcting counts
Generated IQ data


Simulating for distance: 100%|██████████| 15/15 [4:20:42<00:00, 1042.81s/it]
